<a href="https://colab.research.google.com/github/DemirHasicic/AIPortfolio/blob/main/VI_lab2_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ZADATAK 1: Rad u Pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('dataset.csv')
data

In [ ]:
#a)Učitati datoteku kao Pandas dataframe, te prikazati prvih 5 i posljednjih 10 unosa
data.head()
data.tail(10)
#U zagrade se piše koji broj instanci/redova želimo imati prikazane


In [ ]:
#b)Ispisati samo podatke vezane za redovne parcijalne ispite(Ispit1 i Ispit2¸
data.loc[:,'Ispit1']

In [ ]:
data.loc[:,'Ispit2']

In [ ]:
#c)Ispisati sve studente koji su izgubili prisustvo
data.loc[data['Prisustvo']==0]

In [ ]:
#d)Ispisati indeks, ukupne bodove i ocjenu za sve studente sa ocjenom 8 ili viš

data.loc[(data['Ocjena'] =='8')| (data['Ocjena']=='9')| (data['Ocjena']=='10'), ['Indeks', 'UKUPNO','Ocjena']]

In [ ]:
#e)Sve nedostajuce vrijednosti zamijeniti sa vrijednoscu np.nan
data.replace('/', np.nan, inplace=True)

In [ ]:
#f)Izbaciti sve studente koji nemaju ocjenu
bez_ocjena = data[data['Ocjena']=='/'].index
data.drop(bez_ocjena, inplace=True)
data = data.reset_index(drop=True)
 #Pošto se u pozadini .csv reazlizira kao hash mapa nakon izbacivanja nece biti nuzno ocuvan redoslijed

In [ ]:
#g)Kreirati kolonu ispit1_finalu koju ćete za svakog studenta upisati onaj rezultat ispita koji je bolji (na
#primjer, ako je student bolje uradio popravni ispit, onda tu pišete bodove sa popravnog, a u protivnom sa
#redovnog);

data['Ispit1_final'] = np.where(data['Ispit1']>data['Ispit1_popravni'],data['Ispit1'], data['Ispit1_popravni'])
#data['Ispit1_final'] = data['Ispit1']
#data.loc[data['Ispit1'] < data['Ispit1_popravni'],'Ispit1_final'] = data['Ispit1_popravni']
data
#Radi za sve osim za prvi


In [ ]:
data['Ispit2_final'] = np.where(data['Ispit2']>data['Ispit2_popravni'], data['Ispit2'], data['Ispit2_popravni'])
data

In [ ]:
data.drop(['Ispit1','Ispit1_popravni', 'Ispit2', 'Ispit2_popravni'],axis=1, inplace=True)

In [ ]:
data.to_csv('izvjestaj_modificirano.csv', sep=';')


In [ ]:
import pickle

In [ ]:
pickle.dump('izvjestaj_modificirano.csv', open("izvjestaj_modificirano_pickle.p", "wb"), protocol = pickle.HIGHEST_PROTOCOL)



# ZADATAK 2 : Rad u Sklearn


In [ ]:
#a)Ucitati datoteku kao pandas datoteku
data1 = pd.read_csv('izvjestaj_modificirano.csv')

In [ ]:
data1

In [ ]:
#b)Za kolone koje predstavljaju Ispite Nan vrijednosti zamijeniti sa median
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='median')
data.loc[:,'Ispit2'] = si.fit_transform(data.loc[:, 'Ispit2'].values.reshape(-1,1))
data.loc[:, 'Ispit1'] = si.fit_transform(data.loc[:, 'Ispit1'].values.reshape(-1,1))
data

In [ ]:
#c)Za kolone koje predstavljaju ispite Nan vrijednosti zamijeniti sa mean
from sklearn.impute import SimpleImputer
si1 = SimpleImputer(strategy='mean')
data.loc[:, 'Ispit2'] = si.fit_transform(data.loc[:, 'Ispit2'].values.reshape(-1,1))
data.loc[:, 'Ispit1'] = si.fit_transform(data.loc[:, 'Ispit1'].values.reshape(-1,1))
#Obzirom da se radi samo mijenjanje NaN vrijednosti nad jednom kolonom, neophodno je koristiti funkciju reshape,
#jer SimpleImputer očekuje 2D niz. Ukoliko se radi zamjena nad više kolona (ili redova) istovremeno,
#tada se funkcija reshape treba izostaviti.
data.loc[:5,'Ispit2']
data.loc[:5, 'Ispit1']

In [ ]:
#d)Izvrsiti z-score normalizaciju
from sklearn.preprocessing import scale
data['Ispit2'] = scale(data['Ispit2'])
data.loc[:5, 'Ispit2']
data['Ispit1'] = scale(data['Ispit2'])
data.loc[:5, 'Ispit1']

In [ ]:
#e)Izvrisiti MinMax normalizaciju za popravne
from sklearn.preprocessing import MinMaxScaler
mm=MinMaxScaler()
data['Ispit1_popravni'] = mm.fit_transform(data['Ispit1_popravni'].values.reshape(-1,1))
data['Ispit2_popravni'] = mm.fit_transform(data['Ispit2_popravni'].values.reshape(-1,1))
#data.loc[:5, 'Ispit1_popravni']

In [ ]:
#f) Nedostajuće vrijednosti zamijeniti sa nulama
data.replace('/', '0', inplace=True)

In [ ]:
#g) Izdvojiti kolonu ocjena u posebnu varijablu na nacin da ta kolona vise ne postoji
X = data['Ocjena']
data.drop('Ocjena', axis=1, inplace=True)

In [ ]:
#h) Konvertovati originalni skup podataka u numpy niz, kao i varijablu X
Y = data.to_numpy()
X = X.to_numpy()

In [ ]:
#i) Ako X predstavlja labele, a ostatak data predstavlja atribute, izvrisit podjelu na trening i test skup podataka, za trening 80%, za testni 20%
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 42)

In [ ]:
X_train
X_test
Y_train
Y_test

# 'ZADATAK' 3: Klasični algoritam mašinskog učenja

In [ ]:
#a) Ucitali smo podatke za iris iz postojece biblioteke load_iris, te ispisale atribute i nazive labela za taj dataset
from sklearn.datasets import load_iris
iris = load_iris()
x = iris.data
y = iris.target
feature_names = iris.feature_names
target_names  = iris.target_names
print("Nazivi znacajki : ", feature_names)
print("Nazivi labela : ", target_names)
print("\nPrvih 5 redova X :\n", x[:5])

In [ ]:
#b) Podjela skupa podataka na testni i trening skup
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 1)
print(y_train.shape)
print(y_test.shape)
print(x_test.shape)
print(x_train.shape)

(105,)
(45,)
(45, 4)
(105, 4)


In [ ]:
#c) Primjenjujemo odgovarajući algoritam na testni i trening skup u ovom slucaju k - NN , ovaj algoritam svaki podataka mapira kao n-dimenzionalnu tačku. Klasa podatka se zatim
# odredđuje na osnovu k-najblizih tacaka u tom prostoru. Uzima se ona klasa koja se pojavljuje najcesce u blizini k najblizih susjeda
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
classifier_kn = KNeighborsClassifier(n_neighbors = 31)
classifier_kn.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=31)

In [ ]:
#d) Sada algoritam primjenjujemo na testnom skupu, kako bi se odredila uspjesnost, pisemo x_test jer su x podaci iris.data , OPTIMALNA VRIJEDNOST K <=31
y_pred = classifier_kn.predict(x_test)
print("Tacnost :  ", metrics.accuracy_score(y_test, y_pred))

Tacnost :   0.9777777777777777


# ZADATAK 4 : Deep Learning

In [ ]:
#a) Keras ima ucitane podatke slika sa brojevima njih 70000 i vec su podijeljeni na trening i test podatke, samo ih ucitamo iz kerasa
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
#b) Kako bi se ove slike mogle poslati u neuralnu mrezu potrebno ih je pretvoriti u brojeve u pokretnom zarezu, te ih svesti na opseg izmedju 0 i 1
#posto pikseli greyscale slika zauzimaju vrijednosti izmedju 0 i 255 mozemo ih jednostavno podijeliti sa 255 i dobiti trazeni opseg
train_images = x_train.reshape ((x_train.shape[0], 28*28))
train_images = train_images.astype('float32') /255

test_images = x_test.reshape((x_test.shape[0], 28*28))
test_images = test_images.astype('float32')/255
print(test_images.shape)
print(train_images.shape)
print(test_images)


In [ ]:
#c) Nakon što su podaci sprmeni moze se napraviti jednostavan model neuralne mreze. Kreira se model pa se dodaju slojevi
#Model se moze zamisliti kao sito za podatke gdje svaki sloj filtrira podatke (nadamo se na koristan nacin), posljednji sloj je softmax sloj i on ispisuje
#niz vjerovatnoca (cija je suma 1), svaka vrijednost u tom nizu oznacava s kojom vjerovatncom slika pripada odredjenoj cifri gdje cifra predstavlja objekat

In [ ]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation = 'relu', input_shape = (28*28,)))
network.add(layers.Dense(10, activation = 'softmax'))

In [ ]:
#d) Nakon definisanja modela, potrebno ga je i trenirati , kako bi to bilo moguce model se treba i kompajlirati
#kompajliranje se sastoji od odabira 3 parametra 1)funkcije gubitka(Oznaca kako neuralna mreza moze mjeriti svoje performanse, te na osnovu nje prilagodjava procese treniranja)
#2) Optimizator mehanizam kojim se neuralna mreza aktualizira, 3)metrike koje se prate tokom testiranja i treniranja , samo tačnost gledamo / kod funkcije gubitka sve performanse
network.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
#e) Izlaz iz neuralne mreze je niz od 10 elemenata . Shodno tome izvrsavamo pravilno prilagođavanje labele izlazu, izvrsimo one-hot kodiranje istih. Ovo kodiranje predstavlja pretvaranje
#labele u niz od 10 elemenata  npr 3 -> [0 0 0 1 0 0 0 0 0 0] (kao indeksacija 0,1,2,3,4,5,6,7,8,9, pa je na mjestu gdje je 3 stavljena jedinica)
from tensorflow.keras.utils import to_categorical
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)


In [ ]:
#f) Spremno sve da se mreza trenira
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 6s 11ms/step - loss: 0.2591 - accuracy: 0.9249
Epoch 2/5
469/469 [==============================] - 5s 11ms/step - loss: 0.1050 - accuracy: 0.9685
Epoch 3/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0693 - accuracy: 0.9796
Epoch 4/5
469/469 [==============================] - 5s 11ms/step - loss: 0.0497 - accuracy: 0.9853
Epoch 5/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0377 - accuracy: 0.9888


In [ ]:
#g)
test_loss, test_acc = network.evaluate(test_images, test_labels)
train_loss, train_acc = network.evaluate(train_images, train_labels)
print("Tacnost za testni skup: ", test_acc)
print("Tacnost za trening skup: ", train_acc)

#Vidimo da postoji razlika od oko 0.2 izmedju trening i testnog skupa, zadovoljavajuci je rezultat jer je trening skup 0.33 dio testnog skupa

1875/1875 [==============================] - 6s 3ms/step - loss: 0.0261 - accuracy: 0.9926
Tacnost za testni skup:  0.978600025177002
Tacnost za trening skup:  0.9925666451454163
